1) Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, записывающую собранные вакансии в созданную БД

2) Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введенной суммы

3*)Написать функцию, которая будет добавлять в вашу базу данных только новые вакансии с сайта

In [66]:
import requests
from bs4 import BeautifulSoup as bs
from pymongo import MongoClient
from pprint import pprint

import warnings
warnings.filterwarnings('ignore')

In [67]:
class HeadHunter:

    def __init__(self, key):
        self.main_link = 'https://spb.hh.ru/search/vacancy/'
        self.user_agent = {'User-agent': 'Mozilla/5.0'}
        self.key = key

        params = {'text': self.key}
        response = bs(requests.get(self.main_link, headers=self.user_agent, params=params).text, 'lxml')
        self.max_pages = int(response.findAll('a', {'data-qa': 'pager-page'})[-1].text)

    def get_vacancies(self, page):
        params = {'text': self.key, 'page': page}
        response = requests.get(self.main_link, headers=self.user_agent, params=params).text
        return bs(response, 'lxml').findAll('div', {'class': 'vacancy-serp-item'})


    def get_attributes(self, vacancy):

        name = vacancy.find('a', {'data-qa': 'vacancy-serp__vacancy-title'}).text
        
        employer_raw = vacancy.find('a', {'data-qa': 'vacancy-serp__vacancy-employer'})
        employer = employer_raw.text if employer_raw else vacancy.find('div', {'class': 'vacancy-serp-item__meta-info'}).text

        location = vacancy.find('span', {'data-qa': 'vacancy-serp__vacancy-address'}).text
        link = vacancy.find('a', {'data-qa': 'vacancy-serp__vacancy-title'})['href']

        salary_raw = vacancy.find('div', {'class': 'vacancy-serp-item__compensation'})
        salary_dict = {'currency': None, 'min': None, 'max': None}

        if salary_raw:
            if '-' in salary_raw.text:
                salary_dict = {
                    'currency': salary_raw.text.split()[-1] if salary_raw.text.split()[-2] != 'бел.' else 'BYR',
                    'min': int(''.join(salary_raw.text.split()[:-1]).split('-')[0]) if salary_raw.text.split()[-2] != 'бел.' else int(''.join(salary_raw.text.split()[:-2]).split('-')[0]),
                    'max': int(''.join(salary_raw.text.split()[:-1]).split('-')[1]) if salary_raw.text.split()[-2] != 'бел.' else int(''.join(salary_raw.text.split()[:-2]).split('-')[1])
                }
            elif 'от' in salary_raw.text:
                salary_dict = {
                    'currency': salary_raw.text.split()[-1] if salary_raw.text.split()[-2] != 'бел.' else 'BYR',
                    'min': int(''.join(salary_raw.text.split()[1:-1])) if salary_raw.text.split()[-2] != 'бел.' else int(''.join(salary_raw.text.split()[1:-2])),
                    'max': None
                }
            elif 'до' in salary_raw.text:
                salary_dict = {
                    'currency': salary_raw.text.split()[-1] if salary_raw.text.split()[-2] != 'бел.' else 'BYR',
                    'min': None,
                    'max': int(''.join(salary_raw.text.split()[1:-1])) if salary_raw.text.split()[-2] != 'бел.' else int(''.join(salary_raw.text.split()[1:-2]))
                }
            
        return {'placement': 'hh.ru', 
                'name': name, 
                'employer': employer,
                'location': location,
                'link': link, 
                'salary_currency': salary_dict['currency'],
                'salary_min': salary_dict['min'],
                'salary_max': salary_dict['max'],
               }

In [68]:
class SuperJob:

    def __init__(self, key):
        self.main_link = 'https://www.superjob.ru/vacancy/search/'
        self.key = key
        self.user_agent = {'User-agent': 'Mozilla/5.0'}

        self.params = {'keywords': self.key}

        response = bs(requests.get(self.main_link, headers=self.user_agent, params=self.params).text, 'lxml')
        page_list = [page.text for page in response.findAll('a', {'class': '_3ze9n'})]
        self.max_pages = int(page_list[-2]) if page_list else 1

    def get_vacancies(self, page):
        self.params['page'] = page
        response = bs(requests.get(self.main_link, headers=self.user_agent, params=self.params).text, 'lxml')
        return response.findAll('div', {'class': '_3zucV _2GPIV f-test-vacancy-item i6-sc _3VcZr'})

    def get_attributes(self, vacancy):

        name = vacancy.find('div', {'class': '_3mfro CuJz5 PlM3e _2JVkc _3LJqf'}).text
        
        employer = vacancy.find('span', {'class': 'f-test-text-vacancy-item-company-name'}).text    
        location = vacancy.find('span', {'class': 'f-test-text-company-item-location'}).text.split(' • ')[1]
        
        link = 'https://www.superjob.ru' + vacancy.find('a', {'class': 'icMQ_'})['href']

        salary_raw = vacancy.find('span', {'class': 'f-test-text-company-item-salary'}).text.replace('\xa0', ' ')
        salary_dict = {'currency': None, 'min': None, 'max': None}

        if salary_raw != 'По договорённости':
            if '-' in salary_raw:
                salary_dict = {
                    'currency': salary_raw.split()[-1],
                    'min': int(''.join(salary_raw.split()[:-1]).split('-')[0]),
                    'max': int(''.join(salary_raw.split()[:-1]).split('-')[1])
                }
            elif 'от' in salary_raw:
                salary_dict = {
                    'currency': salary_raw.split()[-1],
                    'min': int(''.join(salary_raw.split()[1:3])),
                    'max': None
                }
            elif 'до' in salary_raw:
                salary_dict = {
                    'currency': salary_raw.split()[-1],
                    'min': None,
                    'max': int(''.join(salary_raw.split()[1:3]))
                }

        return {'placement': 'superjob.ru', 
                'name': name,
                'employer': employer,
                'location': location,
                'link': link, 
                'salary_currency': salary_dict['currency'],
                'salary_min': salary_dict['min'],
                'salary_max': salary_dict['max'],
               }

In [69]:
class Mongo:

    def __init__(self, blank=False):
        client = MongoClient('mongodb://127.0.0.1:27017')
        self.items = client['vacancies'].items
        if blank:
            self.items.delete_many({})

    def insert(self, entity):
        self.items.insert_one(entity)

    def head(self, num):
        objects = self.items.find().limit(num)
        for item in objects:
            pprint(item)

    def contains(self, item):
        return self.items.find(item).count()
    
    def prettify(self):
        self.items.update_many({'salary_currency': 'грн.'}, {"$set": {'salary_currency': 'UAH'}})
        self.items.update_many({'salary_currency': 'руб.'}, {"$set": {'salary_currency': 'RUR'}})
        self.items.update_many({'salary_currency': '₽'}, {"$set": {'salary_currency': 'RUR'}})
        self.items.update_many({'salary_currency': 'сум'}, {"$set": {'salary_currency': 'UZS'}})
        currencies = self.items.distinct('salary_currency')
        print(f'Currencies: {list(filter(None, currencies))}')
    
    def salary_greater_then(self, amount, currency=None):
        if currency:
            return self.items.find({
                '$and': [{'salary_currency': currency}, 
                               {'$or': [
                                   {'salary_min':{'$gt': amount}}, 
                                   {'salary_max':{'$gt': amount}}]}]})
        else:
            return self.items.find({'$or': [{'salary_min':{'$gt': amount}}, {'salary_max':{'$gt': amount}}]})
        
            

In [70]:
class Collector:

    def __init__(self, parser, db, key):
        self.key = key
        self.parser = parser(key)
        self.db = db

    def collect(self):
        for page in range(self.parser.max_pages + 1):
            vacancies_list = self.parser.get_vacancies(page)
            for vacancy in vacancies_list:
                item = self.parser.get_attributes(vacancy)
                if not self.db.contains(item):
                    self.db.insert(item)

In [71]:
key = 'пекарь'
mongo = Mongo(blank=True)

for parser in HeadHunter, SuperJob:
    collector = Collector(parser=parser, db=mongo, key=key)
    collector.collect()

In [72]:
mongo.items.count()

2062

In [73]:
mongo.head(1)

{'_id': ObjectId('5e3ae6f0216d2e5baf2acaf7'),
 'employer': ' Перекресток',
 'link': 'https://spb.hh.ru/vacancy/35523841?query=%D0%BF%D0%B5%D0%BA%D0%B0%D1%80%D1%8C',
 'location': 'Дедовск',
 'name': 'Пекарь',
 'placement': 'hh.ru',
 'salary_currency': None,
 'salary_max': None,
 'salary_min': None}


In [74]:
mongo.prettify()

Currencies: ['BYR', 'KZT', 'RUR', 'UAH', 'USD', 'UZS']


In [75]:
max_salaries = mongo.salary_greater_then(50000, currency='RUR')
max_salaries.count()

195

In [76]:
for item in max_salaries.limit(3):
    pprint(item)
    print()

{'_id': ObjectId('5e3ae6f1216d2e5baf2acb0c'),
 'employer': 'ООО ЛавкаЛавка Рестораны',
 'link': 'https://spb.hh.ru/vacancy/35514576?query=%D0%BF%D0%B5%D0%BA%D0%B0%D1%80%D1%8C',
 'location': 'Москва',
 'name': 'Пекарь',
 'placement': 'hh.ru',
 'salary_currency': 'RUR',
 'salary_max': 52900,
 'salary_min': 42000}

{'_id': ObjectId('5e3ae6f1216d2e5baf2acb0f'),
 'employer': ' КрафтенБург',
 'link': 'https://spb.hh.ru/vacancy/35298666?query=%D0%BF%D0%B5%D0%BA%D0%B0%D1%80%D1%8C',
 'location': 'Москва',
 'name': 'Пекарь-кондитер',
 'placement': 'hh.ru',
 'salary_currency': 'RUR',
 'salary_max': 70000,
 'salary_min': 60000}

{'_id': ObjectId('5e3ae6f1216d2e5baf2acb11'),
 'employer': ' Булочные Все Дома',
 'link': 'https://spb.hh.ru/vacancy/35294967?query=%D0%BF%D0%B5%D0%BA%D0%B0%D1%80%D1%8C',
 'location': 'Санкт-Петербург',
 'name': 'Продавец-пекарь',
 'placement': 'hh.ru',
 'salary_currency': 'RUR',
 'salary_max': 52000,
 'salary_min': 35000}

